# Assistant-09<br/>Function calling

## Outlook meeting assistant
- There is no easy way in Outlook today to book a new meeting with the same partipants with an event that already occurred previously. 
- You either have to move the original meeting to a new time (then you loose the meeting history) or you need to manually open the past meeting details, manually copy the participants to a new meeting request window.
- This assistant will scan your local Outlook appointments from last month and it will call an Outlook "New meeting" window with the same participants as you described.

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as helper

# Open Outlook if it is not open
helper.outlook_is_running()

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI)

### Fetch Outlook meetings from last 30 days from your local Outlook

In [2]:
meetings_past_month = helper.get_outlook_meetings(30)


### Prepare the tool for function calling

In [3]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "create_new_meeting",
        "description": "Create a new meeting request with the same participants as another meeting that occurred in the past",
        "parameters": {
            "type": "object",
            "properties": {
                "required_attendees": {
                    "type": "string",
                    "description": "The list of required attendees. Keep the same list of attendees as in the original meeting."
                },
                "subject": {
                    "type": "string",
                    "description": "The list of required attendees. Keep the same subject as the original meeting."
                },
                "optional_attendees": {
                    "type": "string",
                    "description": "The list of optional attendees. Keep the same optiona attendees list as the original meeting."
                }
            },
            "required": ["required_attendees", "subject"]
        }
    }
}]

### Create the assistant

In [4]:
helper.clear_shelves()

## Create an assistant
meeting_assistant = helper.create_assistant(client,
                                                  name="fta-Meeting Assistant",
                                                  instructions="You are an assistant that helps to create new meetings requests in Outlook. You will be provided a list of meetings from last month and you need to setup a new meeting keeping all the same details from the original meeting you found. Only select a single meeting (single JSON) and DO NOT mix data from other meetings.", 
                                                  tools=tools_list, 
                                                  model=helper.gpt_deployment_name)

Added assistant:  asst_MOZaOQBptYcPT4z2AM5JVCUh 1


### Define the function calling delegate

In [5]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "create_new_meeting":
            helper.show_meeting_window(subject=arguments['subject'], required_attendees=arguments['required_attendees'], optional_attendees = arguments['optional_attendees'])
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "OK"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### User input
- The user provide as much details as he remembers from the meeting he want to book a new event with the same participants. Such as: what was the subject and some participants involved?
- The list of last month events will be provded as JSON input and OpenAI will find what is the closest original appointment with the information provided by the user
- And then it will return the function outputs required to call a new Outlook Meeting window locally, which will contains the same Subject and Participants as the original event

In [6]:
userInput = f'I had a meeting with customers from [COMPANY] last week. One of the customer name was [NAME]'

if "[COMPANY]" in userInput or "[NAME]" in userInput:
    raise Exception("Please replace [COMPANY] and [NAME] with your information in line above")

userInput += f'\n\nLast month meetings: ### {meetings_past_month}'

helper.generate_response(client,meeting_assistant, userInput , "123", "Alex", function_calling_delegate)

Creating new thread for Alex with user_id 123
Added thread:  thread_T2GmKj6wvOA1mvVYngP4OWLq 1
Function Calling
{'tool_calls': [{'id': 'call_tqaS20F1rYp8hQAbMjlZfYaf', 'function': {'arguments': '{\n  "required_attendees": "Andre Dewes; ishani.vyas@wellpoint.com; Umar Mohamed Usman; sanjay.durisheti@wellpoint.com",\n  "subject": "[Microsoft FastTrack] Elevance Health working session",\n  "optional_attendees": "Bappaditya Banerjee; Srini Padala; Sanjeev Nair; Ankesh Anupam; Ishani Vyas; Basuki.Winoto@elevancehealth.com; Vyas, Ishani; Narendra Amirineni"\n}', 'name': 'create_new_meeting'}, 'type': 'function'}]}
Submitting outputs back to the Assistant...
Generated message: The new meeting request with the subject "[Microsoft FastTrack] Elevance Health working session" has been created successfully, including all the required and optional attendees as per the original meeting. Please specify a new date and time for this meeting so that it can be scheduled accordingly.
To Alex: The new meet

'The new meeting request with the subject "[Microsoft FastTrack] Elevance Health working session" has been created successfully, including all the required and optional attendees as per the original meeting. Please specify a new date and time for this meeting so that it can be scheduled accordingly.'

### Delete all demo objects

In [7]:
helper.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_MOZaOQBptYcPT4z2AM5JVCUh', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_T2GmKj6wvOA1mvVYngP4OWLq', deleted=True, object='thread.deleted')
Deleting:  0  files.
